In [1]:
import pandas as pd
from glob import glob

In [2]:
ny_files = glob("csv/2019*.csv")
nj_files = glob("csv/JC-2019*.csv")
files = ny_files+nj_files

In [3]:
dfs = []

In [4]:
for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

In [5]:
df = pd.concat(dfs, ignore_index=True)

In [6]:
df = df.drop(columns=["stoptime", "start station name", "start station latitude", "start station longitude", "end station id",
                      "end station name", "end station latitude", "end station longitude", "usertype", "gender"])

In [7]:
date_time = df.starttime.str.rsplit("-", n=1)
date_time

0           [2019-01, 01 00:01:47.4010]
1           [2019-01, 01 00:04:43.7360]
2           [2019-01, 01 00:06:03.9970]
3           [2019-01, 01 00:07:03.5450]
4           [2019-01, 01 00:07:35.9450]
                       ...             
20956639    [2019-12, 31 23:12:18.5480]
20956640    [2019-12, 31 23:19:03.5840]
20956641    [2019-12, 31 23:19:06.5830]
20956642    [2019-12, 31 23:24:00.2630]
20956643    [2019-12, 31 23:45:20.9780]
Name: starttime, Length: 20956644, dtype: object

In [8]:
df["starttime"] = date_time.str[0]+"-01 00:00:00"
df.head()

,tripduration,starttime,start station id,bikeid,birth year
0,320,2019-01-01 00:00:00,3160.0,15839,1971
1,316,2019-01-01 00:00:00,519.0,32723,1964
2,591,2019-01-01 00:00:00,3171.0,27451,1987
3,2719,2019-01-01 00:00:00,504.0,21579,1990
4,303,2019-01-01 00:00:00,229.0,35379,1979


In [9]:
station_df = df.groupby(["start station id", "starttime"]).count()
station_df = station_df.drop(columns = ["tripduration", "birth year"])

In [10]:
station_df = station_df.reset_index()
station_df.columns = ["id", "month", "count"]
station_df.head()

,id,month,count
0,72.0,2019-01-01 00:00:00,1697
1,72.0,2019-02-01 00:00:00,1831
2,72.0,2019-03-01 00:00:00,3075
3,72.0,2019-04-01 00:00:00,4126
4,72.0,2019-05-01 00:00:00,4338


In [ ]:
station_df = station_df.astype({"id":int})
station_df.to_csv("cleaned_csv/station_use.csv", index=False)

In [11]:
df.loc[df["birth year"] < 1940, "birth year"] = "UNK"

In [16]:
birth_cnt_df = df.groupby("birth year").count()
birth_sum_df = df.groupby("birth year").sum()
birth_df = pd.DataFrame({
    "year":birth_cnt_df.index,
    "count": birth_cnt_df.tripduration,
    "duration": birth_sum_df.tripduration
})
birth_df = birth_df.reset_index(drop=True)
birth_df

,year,count,duration
0,1940,4853,3286908
1,1941,6869,6007719
2,1942,9663,7307900
3,1943,9336,8079716
4,1944,8371,6528881
...,...,...,...
60,2000,124819,156601654
61,2001,62622,69588876
62,2002,27200,35402584
63,2003,10357,16106517


In [20]:
birth_df.to_csv("cleaned_csv/birth_year.csv", index=False)

In [18]:
bikeid_cnt_df = df.groupby("bikeid").count()
bikeid_sum_df = df.groupby("bikeid").sum()
bikeid_df = pd.DataFrame({
    "bike_id":bikeid_cnt_df.index,
    "count": bikeid_cnt_df.tripduration,
    "duration": bikeid_sum_df.tripduration
})
bikeid_df = bikeid_df.reset_index(drop=True)
bikeid_df

,bike_id,count,duration
0,14529,675,584342
1,14530,950,750855
2,14531,705,840015
3,14532,11,7799
4,14533,872,810747
...,...,...,...
19926,42044,663,523195
19927,42045,570,427384
19928,42046,614,523935
19929,42068,2,1598


In [21]:
bikeid_df.to_csv("cleaned_csv/bike_id.csv", index=False)

In [22]:
bikeid_df.loc[bikeid_df["bike_id"] == 37947]

,bike_id,count,duration
16013,37947,2,2608974
